In [9]:
from pathlib import Path
import yaml

import pandas as pd

In [10]:
DATA_PATH = Path("../../data/")
OOF_PATH = DATA_PATH / "models"
CONFIG_PATH = Path("../../configs/config.yaml")

models = ["lamau_81425_full_dataset", "lgb_8122_full_dataset",
          "cb_8114_full_dataset", "xgb_81325_full_dataset",
         "lama_81298_full_dataset", "lamann_autoint_8053_full_dataset", "lamann_fttransformer_8050_full_dataset"]

In [11]:
df_train = pd.read_parquet(DATA_PATH / "train_preproc_2.parquet")
df_test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")

## Enrich data

In [13]:
for model in models:
    oof_pred = pd.read_csv(OOF_PATH / model / "oof.csv")
    test_pred = pd.read_csv(OOF_PATH / model / f"{model}.csv")
    
    df_train[model] = oof_pred[model]
    df_test[model] = test_pred["target"]
    break

In [5]:
df_train.to_parquet(DATA_PATH / "train_preproc_oof.parquet", index=False)
df_test.to_parquet(DATA_PATH / "test_preproc_oof.parquet", index=False)

## Add stack columns to the config

In [6]:
with CONFIG_PATH.open() as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)

In [7]:
cfg["stack_features"] = models

In [8]:
with CONFIG_PATH.open("w") as f:
    yaml.dump(cfg, f)